In [1]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-lsp 3.10.2 requires jupyterlab<4.0.0a0,>=3.1.0, which is not installed.


Import the SDK and some helpers for rendering the output.

In [2]:
from google import genai
from google.genai import types

from IPython.display import HTML, Markdown, display

### Set up API key

In [3]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

Explore the Gemini model family.

In [5]:
client = genai.Client(api_key=GOOGLE_API_KEY)
for model in client.models.list():
  print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01

### Few shot prompting

Providing some context or the LLM to understand the task at hand. In this case, we need the output to be in a specific format given some specific data.

In [9]:
few_shot_prompt = """Parse through crowdsourced text, geo-tagged reports on platforms like Ushahidi
that describe what is happening during emergencies, conflicts, elections, or natural disasters. :

EXAMPLE:
Guns shots heard near Toi Market. 3 students caught in the line of fire, ambulances on site.
JSON Response:
```
{
"Type": "gun-violence",
"Location": "Toi Market",
"Impact": "3",
"Response": Ambulances"
}
```

EXAMPLE:
Our house in Uthiru is submerged in flood water. Please help us, a family of six is stranded in the house.
JSON Response:
```
{
"Type": "Floods",
"Location": "Uthiru",
"No of Casualties": "6",
"Response":
}
```


"""

crowdsourced_input = "Mount Etna is quite invisible today. Lots of smoke and tremors near our town in Sicily"

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=250,
    ),
    contents=[few_shot_prompt, crowdsourced_input])

print(response.text)

```json
{
"Type": "Volcanic Activity",
"Location": "Sicily",
"Impact": "Smoke and tremors",
"Response": ""
}
```



### Image understanding
If now in the case that users also uploaded images